In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import gc



## Algorithms

### Hill climbing

In [ ]:
def hill_climbing_move(grid_size,cell_counters,pos):
  x,y = pos
  val = float('inf')
  action = -1

  if x>0 and cell_counters[x-1][y]>val:  # Move up
      action = 0
      val = cell_counters[x-1][y]
  elif x<grid_size[0]-1 and cell_counters[x+1][y]>val:  # Move down
      action = 1
      val = cell_counters[x+1][y]
  elif y>0 and cell_counters[x][y-1]>val:  # Move left
      action = 2
      val = cell_counters[x][y-1]
  elif y<grid_size[1]-1 and cell_counters[x][y+1]>val:  # Move right
      action = 3
      val = cell_counters[x][y+1]
  elif x>0 and y>0 and cell_counters[x-1][y-1]>val:  # Move diagonally up-left
      action = 4
      val = cell_counters[x-1][y-1]
  elif x>0 and y<grid_size[1]-1 and cell_counters[x-1][y+1]>val:  # Move diagonally up-right
      action = 5
      val = cell_counters[x-1][y+1]
  elif x<grid_size[0]-1 and y>0 and cell_counters[x+1][y-1]>val:  # Move diagonally down-left
      action = 6
      val = cell_counters[x+1][y-1]
  elif x<grid_size[0]-1 and y<grid_size[1]-1 and cell_counters[x+1][y+1]>val:  # Move diagonally down-right
      action = 7
      val = cell_counters[x+1][y+1]

  return action

### DQN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from collections import deque

class DQNAgent:
    state_size = 53
    action_size = 0
    memory = deque(maxlen=50)  # Replay buffer
    gamma = 0.95  # Discount factor
    epsilon = 1.0  # Exploration-exploitation trade-off
    epsilon_decay = 0.995
    epsilon_min = 0.01
    learning_rate = 0.001
    epsilon2 = 0.3
    model = None
    target_model = None

    def __init__(self, state_size, action_size):

        # Initialize DQN agent parameters
        self.state_size = 53
        self.action_size = action_size
        self.memory = deque(maxlen=50)  # Replay buffer

        # Build Q-network
        self.model = self.build_model()
        self.target_model = self.build_model()
        self.update_target_model()

    def build_model(self):
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Dense(24, input_dim=self.state_size, activation='relu'))
        model.add(tf.keras.layers.Dense(24, activation='relu'))
        model.add(tf.keras.layers.Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate))
        return model

    def epsilon_greedy(self, state):
        # Epsilon-greedy action selection
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        q_values = self.model.predict(state.reshape(1, -1))
        return np.argmax(q_values[0])

    def update_target_model(self):
      # Update the weights of the target model to match the Q-network weights
        self.target_model.set_weights(self.model.get_weights())

    def update_q_network(self, batch_size):
        # Update Q-network using experience replay
        if len(self.memory) < batch_size:
            return

        minibatch = random.sample(self.memory, batch_size)

        for state, action, reward, next_state in minibatch:
            target = reward + self.gamma * np.amax(self.target_model.predict(next_state.reshape(1, -1))[0])

            target_f = self.model.predict(state.reshape(1, -1))
            target_f[0][action] = target
            self.model.fit(state.reshape(1, -1), target_f, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def remember(self, state, action, reward, next_state):
        # Store experience in replay buffer
        self.memory.append((state, action, reward, next_state))


### Trainer

In [ ]:
class Trainer:
    grid_size = []
    cell_priorities = []
    cell_counters = []
    robot_positions = []
    max_drones = []
    curr_num_drones = 0
    dqn = None

    def __init__(self, grid_size, cell_counters, robot_positions, max_drones):
      self.grid_size = grid_size
      self.cell_counters = np.copy(cell_counters)
      self.robot_positions = np.copy(robot_positions)
      self.max_drones = max_drones
      self.dqn = DQNAgent(3 + grid_size[0]*grid_size[1],8)

    def feed(self,cell_priorities,cell_counters,robot_positions,curr_num_drones):
      self.cell_priorities = cell_priorities
      self.cell_counters = cell_counters
      self.robot_positions = np.copy(robot_positions)
      self.curr_num_drones = curr_num_drones

    def reset(self, cell_counters, robot_positions, max_drones):
      self.cell_counters = np.copy(cell_counters)
      self.robot_positions = np.copy(robot_positions)
      self.max_drones = max_drones
      state = self.get_state()
      return state

    def step(self, robot_position , action):

        x, y = robot_position

        # Update the timers based on the action
        if action == 0:  # Move up
            x = max(0, x - 1)
        elif action == 1:  # Move down
            x = min(self.grid_size[0] - 1, x + 1)
        elif action == 2:  # Move left
            y = max(0, y - 1)
        elif action == 3:  # Move right
            y = min(self.grid_size[1] - 1, y + 1)
        elif action == 4:  # Move diagonally up-left
            x = max(0, x - 1)
            y = max(0, y - 1)
        elif action == 5:  # Move diagonally up-right
            x = max(0, x - 1)
            y = min(self.grid_size[1] - 1, y + 1)
        elif action == 6:  # Move diagonally down-left
            x = min(self.grid_size[0] - 1, x + 1)
            y = max(0, y - 1)
        elif action == 7:  # Move diagonally down-right
            x = min(self.grid_size[0] - 1, x + 1)
            y = min(self.grid_size[1] - 1, y + 1)

        reward = 0

        if self.cell_counters[x][y] == 0:
          reward = 1.0
        elif self.cell_counters[x][y] < 0:
          reward = self.cell_counters[x][y] / 20
        else:
          reward = 1.0 / (self.cell_counters[x][y] + 1)

        robot_position = (x, y)
        next_state = self.get_state(robot_position)
        return next_state, reward

    def get_state(self, robot_position):
      state = []

      x, y = robot_position

      state.append(self.curr_num_drones)
      state.append(x)
      state.append(y)

      zone_size = 10
      num_rows = int(self.grid_size[0]/zone_size)
      num_cols = int(self.grid_size[1]/zone_size)
      for i in range(num_rows):
        for j in range(num_cols):
            lateness = 0
            for x in range(i * zone_size , (i + 1) * zone_size):
              for y in range(j * zone_size , (j + 1) * zone_size):
                if self.cell_counters[x][y] < 0:
                  lateness += abs(self.cell_counters[x][y])
            state.append(lateness)


      return state

    def train(self, update_target_flag , robot_positions):
        actions = []
        rewards = []
        total_reward = 0
        for pos in robot_positions:
            state = self.get_state(pos)
            state = np.asarray(state).astype(np.float32)
            action = self.dqn.epsilon_greedy(state)
            actions.append(action)

            next_state, reward = self.step(pos, action)

            total_reward += reward

            rewards.append(reward)

            next_state = np.asarray(next_state).astype(np.float32)

            self.dqn.remember(state, action, reward, next_state)

        self.dqn.update_q_network(batch_size=32)

        # Periodically update the target model weights
        if update_target_flag == True:
            self.dqn.update_target_model()

        # global store_rewards

        # store_rewards.append(total_reward)
        print(f" Total Reward: {total_reward}")
        return actions


## Drone class

In [ ]:
class Drone:
    drone_id = 0
    current_pos = None
    cell_counters = []
    last_comm_time = 0
    neighbours = []
    battery_left = 0
    min_battery_threshold = 0
    grid_size = []
    non_directed_action_probability = 0.1

    def __init__(self, drone_id, pos, last_comm_time, cell_counters, battery_left, min_battery_threshold, grid_size):
        self.drone_id = drone_id
        self.current_pos = pos
        self.last_comm_time = last_comm_time
        self.cell_counters = np.copy(cell_counters)
        self.battery_left = battery_left
        self.min_battery_threshold = min_battery_threshold
        self.grid_size = grid_size


    def __eq__(self, other):
        if isinstance(other, Drone):
            return self.drone_id == other.drone_id
        return False

    def update_cell_priorities(self, counters):
        for i in range(self.grid_size[0]):
          for j in range(self.grid_size[1]):
            self.cell_counters[i][j]-=1
        for p in counters:
            self.cell_counters[p[0]][p[1]] = p[2]

    def drain_battery(self, amount):
        self.battery_left -= amount
        if self.battery_left < self.min_battery_threshold:
            return -1
        else:
            return 1

    def move(self,given_move):
        x, y = self.current_pos
        if given_move == 0:  # Move up
            x = max(0, x - 1)
        elif given_move == 1:  # Move down
            x = min(self.grid_size[0] - 1, x + 1)
        elif given_move == 2:  # Move left
            y = max(0, y - 1)
        elif given_move == 3:  # Move right
            y = min(self.grid_size[1] - 1, y + 1)
        elif given_move == 4:  # Move diagonally up-left
            x = max(0, x - 1)
            y = max(0, y - 1)
        elif given_move == 5:  # Move diagonally up-right
            x = max(0, x - 1)
            y = min(self.grid_size[1] - 1, y + 1)
        elif given_move == 6:  # Move diagonally down-left
            x = min(self.grid_size[0] - 1, x + 1)
            y = max(0, y - 1)
        elif given_move == 7:  # Move diagonally down-right
            x = min(self.grid_size[0] - 1, x + 1)
            y = min(self.grid_size[1] - 1, y + 1)
        self.current_pos = (x,y)
        return (x,y)

    ## Need to update this  (Calculate lateness here too for final evaluation)
    def decide_and_move(self,algo, curr_num_drones):

      state = []

      x, y = self.current_pos

      state.append(curr_num_drones)
      state.append(x)
      state.append(y)

      zone_size = 10
      num_rows = int(self.grid_size[0]/zone_size)
      num_cols = int(self.grid_size[1]/zone_size)
      for i in range(num_rows):
        for j in range(num_cols):
            lateness = 0
            for x in range(i * zone_size , (i + 1) * zone_size):
              for y in range(j * zone_size , (j + 1) * zone_size):
                if self.cell_counters[x][y] < 0:
                  lateness += abs(self.cell_counters[x][y])
            state.append(lateness)


      state = np.asarray(state).astype(np.float32)
      if np.random.rand() <= self.non_directed_action_probability:
        res = random.randrange(8)
      else:
        res = algo(state)
      return self.move(res)

    def decide_and_move_hill_climb(self):
        if np.random.rand() <= self.non_directed_action_probability:
          res = random.randrange(8)
        else:
          res = hill_climbing_move(self.grid_size,self.cell_counters,self.current_pos)
        return self.move(res)

## Environment class

In [ ]:
class Environment:
    flying_drones = []
    grid_size = []

    def reset(self):
      self.flying_drones = []

    def suspicious_locations(self):
        max_row = self.grid_size[0]
        max_col = self.grid_size[1]
        mean = 0
        std_dev = 5
        num_samples = self.flying_drones.__len__()
        suspicious_values = np.random.normal(mean, std_dev, num_samples)
        return suspicious_values

    def __init__(self, grid_size):
        self.grid_size = grid_size

    def add_drone(self,drone):
        self.flying_drones.append(drone)

    def drain_battery(self):
        to_remove = []
        for drone in self.flying_drones:
            res = drone.drain_battery(random.uniform(1.0, 2.0))
            if res == -1:
              to_remove.append(drone)
        for drone in to_remove:
          self.flying_drones.remove(drone)
        return to_remove

## Basestation class

In [ ]:
class Basestation:
  grid_size = []
  next_drone_id = 0
  cell_priorities = []
  cell_counters = []
  flying_drones = []
  robot_movement_data = []
  suspicion_data = []
  pos = (0,0)
  trainer = None

  def reset(self):
    self.next_drone_id = 0
    for i in range(self.grid_size[0]):
      for j in range(self.grid_size[1]):
        self.cell_counters[i][j] = self.cell_priorities[i][j]
    self.suspicion_data = []
    self.flying_drones = []
    self.robot_movement_data = []

  def __init__(self, grid_size, max_drones):
    self.grid_size = grid_size
    self.cell_priorities = np.random.randint(10, 25, size=(grid_size[0], grid_size[1]))  # Random initial timers
    self.cell_counters = np.copy(self.cell_priorities)
    self.trainer = Trainer(grid_size,self.cell_priorities,self.robot_movement_data,max_drones)

  def receive_data(self,robot_movement_data, suspicion_data):
    self.robot_movement_data = np.copy(robot_movement_data)
    self.suspicion_data = suspicion_data
    total_lateness = 0
    for x,y in robot_movement_data:
      if self.cell_counters[x][y] < 0:
        total_lateness += abs(self.cell_counters[x][y])
    return total_lateness

  def fly_new_drone(self,curr_time):
    drone = Drone(drone_id=self.next_drone_id,
                  pos = self.pos,
                  last_comm_time=curr_time,
                  cell_counters=self.cell_counters,
                  battery_left=random.randint(30, 50),
                  min_battery_threshold=10,
                  grid_size=self.grid_size)
    self.flying_drones.append(drone)
    arrayTemp = np.array([self.pos], dtype=tuple)
    if self.next_drone_id==0:
      self.robot_movement_data  = np.array([self.pos], dtype=tuple)
    else:
      self.robot_movement_data = np.concatenate((self.robot_movement_data, arrayTemp),axis=0)
    self.next_drone_id+=1
    return drone

  def broadcast_and_update_prioities(self):
    changed = []
    for i in range(self.grid_size[0]):
      for j in range(self.grid_size[1]):
        self.cell_counters[i][j]-=1
    robo = 0
    for d in self.suspicion_data:
      new_priority = 1 + 4 * d
      x,y = self.flying_drones[robo].current_pos
      self.cell_priorities[x][y] = min(new_priority,25)
      robo+=1
    for d in self.robot_movement_data:
      self.cell_counters[d[0]][d[1]] = self.cell_priorities[d[0]][d[1]]
      changed.append([d[0],d[1],self.cell_priorities[d[0]][d[1]]])
    return changed

  def operate_trainer(self,clock):
    self.trainer.feed(self.cell_priorities,self.cell_counters,self.robot_movement_data,self.flying_drones.__len__())
    return self.trainer.train(clock%10,self.robot_movement_data)

  def end_simulation(self,robot_movement_data, suspicion_data):
    total_lateness = self.receive_data(robot_movement_data, suspicion_data)
    self.broadcast_and_update_prioities()
    for x in range(self.grid_size[0]):
      for y in range(self.grid_size[1]):
        if self.cell_counters[x][y] < 0:
          total_lateness += abs(self.cell_counters[x][y])
    return total_lateness

## Simulator

In [ ]:
class Simulator:
  clock = 0
  simulation_time = 100
  warmup_time = 0
  max_drones = 0
  grid_size = []
  flying_drones = []
  to_broadcast = []
  robot_movement_data = []
  suspicion_data = []
  new_drones = []
  environment = None
  basestation = None
  drone_movement_algo = None
  is_training = False
  predicted_moves_trainer = []

  total_lateness = 0

  def __init__(self,simulation_time,warmup_time,max_drones,grid_size):
    self.simulation_time = simulation_time
    self.warmup_time = warmup_time
    self.max_drones = max_drones
    self.grid_size = grid_size
    self.environment = Environment(grid_size)
    self.basestation = Basestation(grid_size,max_drones)

  def operate_basestation(self):
    self.total_lateness += self.basestation.receive_data(self.robot_movement_data, self.suspicion_data)
    self.to_broadcast = self.basestation.broadcast_and_update_prioities()
    if(self.flying_drones.__len__()<self.max_drones):
      self.new_drones.append(self.basestation.fly_new_drone(self.clock))
    self.flying_drones += self.new_drones
    if self.is_training:
      self.predicted_moves_trainer = self.basestation.operate_trainer(self.clock)

  def operate_drones(self):
    for drone in self.flying_drones:
      drone.update_cell_priorities(self.to_broadcast)
    if self.is_training:
      for i in range(self.flying_drones.__len__()):
        movement = self.flying_drones[i].move(self.predicted_moves_trainer[i])
        if movement!=None:
          self.robot_movement_data.append(movement)
    else:
      for drone in self.flying_drones:
        movement = drone.decide_and_move(self.drone_movement_algo, len(self.flying_drones))
        # movement = drone.decide_and_move_hill_climb()
        if movement!=None:
          self.robot_movement_data.append(movement)

  def operate_environment(self):
    robots_to_remove = self.environment.drain_battery()
    self.environment.flying_drones += self.new_drones
    for drone in self.flying_drones:
      if drone in robots_to_remove:
        self.flying_drones.remove(drone)
        self.basestation.flying_drones.remove(drone)
    self.suspicion_data = self.environment.suspicious_locations()

  def run_simulation(self):
    if self.warmup_time!=0:
      self.is_training = True
    while self.clock < self.warmup_time:
      if self.clock%30==0:
        gc.collect()
      self.operate_basestation()
      self.robot_movement_data = []
      self.suspicion_data = []
      self.operate_drones()
      self.operate_environment()
      self.new_drones = []
      self.clock+=1

    self.drone_movement_algo = self.basestation.trainer.dqn.epsilon_greedy
    self.is_training = False
    self.total_lateness = 0
    self.flying_drones = []
    self.to_broadcast = []
    self.robot_movement_data = []
    self.suspicion_data = []
    self.new_drones = []
    self.predicted_moves_trainer = []
    self.environment.reset()
    self.basestation.reset()
    self.clock = 0

    while self.clock < self.simulation_time:
      self.operate_basestation()
      self.robot_movement_data = []
      self.suspicion_data = []
      self.operate_drones()
      self.operate_environment()
      self.clock+=1
      self.new_drones = []
    self.total_lateness = self.basestation.end_simulation(self.robot_movement_data, self.suspicion_data)

simulator = Simulator(200,300,35,[100,50])
simulator.run_simulation()
print(f"Total Lateness: {simulator.total_lateness}")

